FRED data: 
- VIX (volatility index): Measures market fear/uncertainty. Spikes during crises.
- 10Y treasury: 
- 2Y treasury:
- Yield curve: (10Y - 2Y) Predicts economic cycles. Inversion often precedes recessions
- BAA corporate Bond: Captures perceived credit risk in corporate bonds. Widens during financial stress.
- CPI: Indicates inflationary environment. Influences policy and real returns.
- Unemployment rate: Reflects slack in the labour market. Lags the business cycle but still valuable.
- Fed Funds Rate: Measures monetary policy stance. Sharp changes can trigger regime shifts.
- GDP growth: measures real economic activity (however it is in quarterly not monthly so may have to forward fill to each corresponding month i.e. fill each missing month with the most recent known GDP value)
- Credit spread (BAA - GS10) (calculated)
- YoY Inflation = 12-month % change in CPI 

In [1]:
import pandas as pd
from datetime import datetime
from fredapi import Fred
from dotenv import load_dotenv
import os


In [2]:
load_dotenv()

fred_api_key = os.getenv("FRED_API_KEY")

f = Fred(api_key=fred_api_key) # Instantiate FRED with API key 


In [3]:
# Define series of interest
series = {
    'VIX': 'VIXCLS',
    '10Y': 'GS10',
    '2Y': 'GS2',
    'YieldCurve': 'T10Y2Y',
    'BAA': 'BAA', # Corporate bond yield (credit risk)
    'CPI': 'CPIAUCSL',
    'UnemploymentRate': 'UNRATE',
    'FedFundsRate': 'FEDFUNDS',
    'GDP_YoY': 'A191RL1Q225SBEA'
}

In [4]:
start_date = '1963-07-01'
end_date = '2025-05-31'


In [5]:
macro_data = {}
for name, code in series.items():
    macro_data[name] = f.get_series(code, observation_start=start_date, observation_end=end_date)


In [6]:
macro_df = pd.DataFrame(macro_data) # Convert each to a dataframe


In [7]:
macro_df['Date'] = macro_df.index # Add date as a column


In [8]:
print(macro_df.head())  # Display the first few rows of the dataframe
print(macro_df.tail())  # Display the last few rows of the dataframe

            VIX   10Y  2Y  YieldCurve   BAA    CPI  UnemploymentRate  \
1963-07-01  NaN  4.02 NaN         NaN  4.84  30.69               5.6   
1963-08-01  NaN  4.00 NaN         NaN  4.83  30.75               5.4   
1963-09-01  NaN  4.08 NaN         NaN  4.84  30.72               5.5   
1963-10-01  NaN  4.11 NaN         NaN  4.83  30.75               5.5   
1963-11-01  NaN  4.12 NaN         NaN  4.84  30.78               5.7   

            FedFundsRate  GDP_YoY       Date  
1963-07-01          3.02      9.1 1963-07-01  
1963-08-01          3.49      NaN 1963-08-01  
1963-09-01          3.48      NaN 1963-09-01  
1963-10-01          3.50      2.6 1963-10-01  
1963-11-01          3.48      NaN 1963-11-01  
              VIX  10Y  2Y  YieldCurve  BAA  CPI  UnemploymentRate  \
2025-05-26  20.57  NaN NaN         NaN  NaN  NaN               NaN   
2025-05-27  18.96  NaN NaN        0.51  NaN  NaN               NaN   
2025-05-28  19.31  NaN NaN        0.51  NaN  NaN               NaN   
2025-

In [9]:
# Compute derived columns
macro_df['CreditSpread'] = macro_df['BAA'] - macro_df['10Y']
macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100
macro_df['GDP_YoY'] = macro_df['GDP_YoY'].ffill()


C:\Users\User\AppData\Local\Temp\ipykernel_22256\1166655369.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100


In [11]:
# Reset index and reorder
macro_df.reset_index(drop=True, inplace=True)
macro_df = macro_df[['Date', 'VIX', '2Y', '10Y', 'YieldCurve', 'CreditSpread',
                 'FedFundsRate', 'Inflation_YoY', 'UnemploymentRate', 'GDP_YoY']]


Now need to make it monthly in line with factor data

In [12]:
macro_df.set_index('Date', inplace=True)
monthly_macro = macro_df.resample('MS').mean()
monthly_macro.reset_index(inplace=True)


In [ ]:
monthly_macro.to_csv('monthly_macro.csv', index=False)
all_monthyl_macro.to_csv('all_monthly_macro.csv', index=False)